In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
cd ml-100k

C:\Users\tyadav\Downloads\ml-100k


In [3]:
# Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

In [4]:
# Reading rating files:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings= pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')

In [5]:
# Reading Item file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

In [6]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [82]:
top = [704, 482, 512, 488 ,1100]
items.iloc[0]

movie id                                                              1
movie title                                            Toy Story (1995)
release date                                                01-Jan-1995
video release date                                                  NaN
IMDb URL              http://us.imdb.com/M/title-exact?Toy%20Story%2...
unknown                                                               0
Action                                                                0
Adventure                                                             0
Animation                                                             1
Children's                                                            1
Comedy                                                                1
Crime                                                                 0
Documentary                                                           0
Drama                                                           

In [84]:
for j in top:
    print(items.iloc[j])

movie id                                                            705
movie title                                  Singin' in the Rain (1952)
release date                                                01-Jan-1952
video release date                                                  NaN
IMDb URL              http://us.imdb.com/M/title-exact?Singin'%20in%...
unknown                                                               0
Action                                                                0
Adventure                                                             0
Animation                                                             0
Children's                                                            0
Comedy                                                                0
Crime                                                                 0
Documentary                                                           0
Drama                                                           

In [113]:
"""
Six Degrees of Separation (1993), -- Drama, Mystery
Notorious (1946), -- Film Noir, Romance, Thriller
Third Man, The (1949), -- Mystery, Thriller
Casablanca (1942), -- Drama, Romance, War
Singin in the Rain (1952), -- Musical, Romance
"""

In [110]:
"""
Toy Story (1995), -- Animation, Children, Comedy
Four Rooms (1995), -- Thriller
Shanghai Triad (Yao a yao yao dao waipo qiao) ..., -- Drama
Twelve Monkeys (1995), -- Drama, Sci-Fi
Dead Man Walking (1995) -- Drama, 
"""

[1, 3, 6, 7, 9]

In [111]:
items.iloc[0], items.iloc[2], items.iloc[5],items.iloc[6],items.iloc[8]

(movie id                                                              1
 movie title                                            Toy Story (1995)
 release date                                                01-Jan-1995
 video release date                                                  NaN
 IMDb URL              http://us.imdb.com/M/title-exact?Toy%20Story%2...
 unknown                                                               0
 Action                                                                0
 Adventure                                                             0
 Animation                                                             1
 Children's                                                            1
 Comedy                                                                1
 Crime                                                                 0
 Documentary                                                           0
 Drama                                             

In [7]:
class MF():
    # Initializing the user-movie rating matrix,
    # number of latent features, alpha and beta
    
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        
    # Initializing user-features and movie-feature matrix
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size = (self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size = (self.num_items, self.K))
        
        # Initializing the bias terms
        
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        
        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
            ]
        # Stochastic gradient descent for given number of iterations
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 20 == 0:
                print("Iteration: %d ; error = %.4f" %(i+1, mse))
            
        return training_process
    
    # Computing total mean squared error
    
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)
    
    # Stochastic Gradient Descent to get optimized P and Q matrix
    
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r-prediction)
            
            self.b_u[i] += self.alpha * (e-self.beta*self.b_u[i])
            self.b_i[j] += self.alpha * (e-self.beta*self.b_i[i])
            
            self.P[i,:] += self.alpha*(e*self.Q[j,:] - self.beta*self.P[i,:])
            self.Q[j,:] += self.alpha*(e*self.P[i,:] - self.beta*self.Q[j,:])
            
            
    # Rating for user i and movie j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i,:].dot(self.Q[j,:].T)
        return prediction
    
    # Full user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis,:] + mf.P.dot(mf.Q.T)

In [8]:
R= np.array(ratings_train.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0))


In [92]:
R[0]

array([5., 3., 4., ..., 0., 0., 0.])

In [9]:
mf = MF(R, K=k, alpha=0.01, beta=0.01, iterations=100)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()

Iteration: 20 ; error = 212.0902
Iteration: 40 ; error = 103.1755
Iteration: 60 ; error = 72.8269
Iteration: 80 ; error = 61.7377
Iteration: 100 ; error = 56.3031

P x Q:
[[4.50751453 2.86488706 3.9574794  ... 3.21783296 3.47211308 3.35889766]
 [4.0751039  3.43754862 2.69944174 ... 3.25349102 3.63181374 3.51312593]
 [3.34827358 2.82550993 2.50862487 ... 2.53925261 2.88929959 2.81571138]
 ...
 [4.97284502 3.4457824  3.51988877 ... 3.50981011 3.82942816 3.74620647]
 [4.53688704 4.26815163 4.96368721 ... 3.75131987 3.94358313 3.8616093 ]
 [2.77426696 4.86807975 2.74903754 ... 2.95362612 3.2818613  3.27168784]]



In [10]:
op = mf.full_matrix()

In [56]:
reco = []
reco_rate = []
for i in range(len(op[0])):
    if op[0][i] > 4 and op[0][i] < 5 and R[0][i] == 0:
        reco.append(i)
        reco_rate.append(op[0][i])

In [59]:
recommendation = pd.DataFrame([reco, reco_rate])

In [62]:
recommendation = recommendation.T

In [65]:
recommendation

,0,1
0,274.0,4.536846
1,275.0,4.442143
2,278.0,4.421320
3,286.0,4.126441
4,296.0,4.839832
5,302.0,4.261990
6,303.0,4.459133
7,305.0,4.521580
8,312.0,4.627284
9,314.0,4.671008


In [70]:
recommend = recommendation.sort_values(by =1, ascending = 0) 

In [73]:
recommend[0].head()

87      704.0
35      482.0
47      512.0
38      488.0
144    1100.0
Name: 0, dtype: float64

In [99]:
likes = []
for i in range(len(R[0])):
    if R[0][i]>=4 and R[0][i]<=5:
        likes.append(i+1)

In [16]:
pred = []
for i in range(len(ratings_test)):
    pred.append(op[ratings_test.user_id[i]-1, ratings_test.movie_id[i]-1])
pred = np.array(pred)
print(np.mean(np.round(pred) == ratings_test.rating))
from sklearn.metrics import precision_score
print(precision_score( ratings_test.rating, np.round(pred), average='macro'))

0.8274696427838254